In [4]:
### 1. 여러 학습 알고리즘에서 최선의 모델 선택하기
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# 랜덤 시드를 설정
np.random.seed(0)

# 데이터를 로드
iris=datasets.load_iris()
features, target=iris.data, iris.target

# 파이프 라인
pipe=Pipeline([('classifier', RandomForestClassifier())])

# 후보 학습 알고리즘과 하이퍼파라미터 지정
search_space=\
    [{'classifier': [LogisticRegression(max_iter=1000)], "classifier__penalty": ['l1','l2'], 'classifier__C': np.logspace(0,4,10)},\
    {'classifier': [RandomForestClassifier()], "classifier__n_estimators": [10,100,1000], 'classifier__max_features': [1,2,3]}]

# 그리드 서치 객체
gridsearch=GridSearchCV(pipe, search_space, cv=5, verbose=0)

# 그리드 서치를 수행
best_model=gridsearch.fit(features, target)

D:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\anaconda3\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The scor

In [5]:
# 최선의 모델을 확인
best_model.best_estimator_

Pipeline(steps=[('classifier',
                 LogisticRegression(C=21.544346900318832, max_iter=1000))])

In [6]:
# 타겟 벡터를 예측
best_model.predict(features)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [7]:
### 2. 전처리와 함께 최선의 모델 선택하기
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pipe=Pipeline([("std", StandardScaler()), ('pca',PCA()), ('classifier', LogisticRegression())])

# 후보 값을 정의
search_space=[{"pca__n_components": [1,2,3], "classifier__penalty": ['l1','l2'], "classifier__C": np.logspace(0,4,10)}]

# 그리드 서치 객체 
clf=GridSearchCV(pipe, search_space, cv=5, n_jobs=-1)

# 그리드 서치를 수행
best_model=clf.fit(features, target)

D:\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan 0.92       0.91333333 0.96
        nan        nan        nan 0.92       0.90666667 0.96666667
        nan        nan        nan 0.92       0.90666667 0.97333333
        nan        nan        nan 0.92       0.90666667 0.97333333
        nan        nan        nan 0.92       0.90666667 0.97333333
        nan        nan        nan 0.92       0.90666667 0.96666667
        nan        nan        nan 0.92       0.90666667 0.96666667
        nan        nan        nan 0.92       0.90666667 0.96666667
        nan        nan        nan 0.92       0.90666667 0.96666667
        nan        nan        nan 0.92       0.90666667 0.96666667]
  warnings.warn(


In [8]:
clf.best_score_

0.9733333333333334

In [9]:
clf.best_estimator_

Pipeline(steps=[('std', StandardScaler()), ('pca', PCA(n_components=3)),
                ('classifier', LogisticRegression(C=7.742636826811269))])

In [10]:
# 최선의 주성분 개수를 확인
clf.best_estimator_.get_params()['pca__n_components']

3

In [13]:
### 3. 알고리즘에 특화된 기법을 사용하여 모델 선택 수행 속도 높이기

# 각 모형별로 특화된 교차검증 하이퍼 파라미터 튜닝 방법

from sklearn import linear_model, datasets

# 데이터를 로드
iris=datasets.load_iris()
features, target=iris.data, iris.target

# 로지스틱 회귀 모형에 특화된 교차검증 하이퍼 파라미터 튜닝 클래스
# Cs : Cost values 100개의 Cost를 튜닝
logit=linear_model.LogisticRegressionCV(max_iter=1000, Cs=100)

# 모델을 훈련
logit.fit(features, target)

LogisticRegressionCV(Cs=100, max_iter=1000)

In [15]:
print(logit.score(features, target))
print(logit.C_) # 최적의 C

0.9866666666666667
[5.85702082 5.85702082 5.85702082]
